In [29]:
using Pkg; Pkg.activate(".");
Pkg.precompile()
using Plots; plotlyjs();
using FourierAnalysis, FFTW, DSP, CSV, Tables;

 Activating environment at `~/Documents/thesis.git/coding/01_exploration/Project.toml`
Precompiling project...


In [89]:
ecg_matrix = CSV.File("../mitbih/100.mitbih") |> Tables.matrix;

fs = 360     # 360 samples per second
N  = 20 * fs  # equivalent to 2 seconds

ecg = ecg_matrix[1:N, 2];
digital_filter = digitalfilter(Lowpass(0.2), Butterworth(2))
ecg_filtered = copy(ecg)
ecg_filtered[:,:] = filt(digital_filter, ecg[:,:]);

In [90]:
# using FFTW.jl only

#Σ = abs.(P * ecg)

#p1 = plot(Σ[brange(N, DC=false), :], labels="fftw raw", xlim=(0, 100));

In [91]:
# using FourierAnalysis.jl
Σ = spectra(ecg, fs, N; tapering=rectangular, func=√, DC=false)

#p2 = plot(abs.(Σ.y[brange(N, DC=false), :]), labels="fa raw", xlim=(0,100));
plot(abs.(Σ.y[brange(N, DC=false), :]), labels="fa raw", xlim=(0,100));

In [92]:
# using FFTW.jl only
#P = plan_rfft(ecg, 1) * (2 / N);
#Σ = abs.(P * ecg)

#p3 = plot(Σ[brange(N, DC=false), :], labels="fftw filtered", xlim=(0, 100));

In [103]:
# using FourierAnalysis.jl
Σ2 = spectra(ecg_filtered, fs, N; tapering=rectangular, func=√, DC=false)

#p4 = plot(abs.(Σ2.y[brange(N, DC=false), :]), labels="fa filtered", xlim=(0,100));
plot(Σ.y[brange(N, DC=false), :], labels="fa raw")
#plot!(frequencies, Σ2.y[brange(N, DC=false), :], labels="fa filtered")

<!DOCTYPE html>
 
 
 Plots.jl

In [94]:
using HeartRateVariability

In [95]:
ecg = ecg_matrix[1:N, 1:2];

In [96]:
time_domain(ecg[:,2])

(mean = -0.31916527777777776, sdnn = 0.1685209072982246, rmssd = 0.04979402636676552, sdsd = 0.046228374684141954, nn50 = 0, pnn50 = 0.0, nn20 = 0, pnn20 = 0.0, rRR = NaN)

In [97]:
frequency(ecg[:,2])

(vlf = 0.0, lf = 0.0, hf = 0.001919080019477814, lfhf_ratio = 0.0, tp = 0.001919080019477814)